In [11]:
import numpy as np
import pickle
import glob
import os

import neuraltda.topology2 as tp2
import pandas as pd
from joblib import Parallel, delayed
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import pickle
import glob
import os
from ephys import core, events, clust
import numpy as np
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
scgFold = '/home/brad/emily/P01S15/scg/'

In [4]:
scgfOld = glob.glob(os.path.join(scgFold, '*-Targ-correct-wide.scg'))[0]
print(scgfOld)

IndexError: list index out of range

In [5]:
scgfNew = glob.glob(os.path.join(scgFold, '*-Targ-correct-wide.scg'))[0]
print(scgfNew)

/home/brad/emily/P01S15/scg/20170516T175952Z-10.0-5.0-13.0-Targ-correct-wide.scg


In [ ]:
with open(scgfOld, 'r') as f:
    scgOld = pickle.load(f)
    

In [6]:
with open(scgfNew, 'r') as f:
    scgNew = pickle.load(f)

In [10]:
len(scgNew['T3S3D40'])

29

In [14]:
# Bin The Data

# Correct/Incorrect
winSizes = [5.0, 10.0, 25.0, 50.0]
segmentInfo = [0, 0] # Sample + Target/Distractor period
#segmentInfo = [2500, 0] # Target+Distractor period
povers = 0.5

blockPath='/home/brad/emily/P01S15/'

winSize = 10.0 #ms
thresh = 13.0
povers = 0.5
cluster_group = ['Good', 'MUA']
widenarrow_threshold = 0.000230 # sw threshold in seconds
segmentInfo = [2500, 0] # SD period
spikes = core.load_spikes(blockPath)
trials = events.load_trials(blockPath)
fs = core.load_fs(blockPath)

#Get Wide/Narrow Clusters
clusters = core.load_clusters(blockPath)
clusters_list = clusters[clusters.quality.isin(cluster_group)]['cluster'].unique()

(wide, narrow) = clust.get_wide_narrow(blockPath, clusters_list, widenarrow_threshold)

# Get Passive Trials
passiveTrials = trials[-(trials['correct'].isin([True, False]))]

In [15]:
passiveTrials

,recording,time_samples,stimulus,stimulus_end,response,response_time,consequence,correct,datetime,correction
34,13,50001,T3S8D-1,162500,N,275000.0,NaN,NaN,2016-07-05 13:01:57.994400,True
220,30,50001,T3S3D40,162501,N,275001.0,NaN,NaN,2016-07-05 13:04:51.187560,True
515,99,50000,T3S3D-1,162500,NaN,NaN,NaN,NaN,2016-07-05 13:15:43.638640,True
2,100,50001,T-1S-1D40,100001,NaN,NaN,NaN,NaN,2016-07-05 13:15:51.895240,True
3,101,50001,T3S8D-1,162501,NaN,NaN,NaN,NaN,2016-07-05 13:15:57.404400,True
4,102,50001,T3S3D40,162501,NaN,NaN,NaN,NaN,2016-07-05 13:16:05.874800,True
5,103,50001,T3S3D40,162501,NaN,NaN,NaN,NaN,2016-07-05 13:16:14.103680,True
6,104,50001,T3S8D40,162501,NaN,NaN,NaN,NaN,2016-07-05 13:16:21.854920,True
7,105,50001,T3S8D40,162501,NaN,NaN,NaN,NaN,2016-07-05 13:16:30.150960,True
8,106,50001,T-1S-1D40,100001,NaN,NaN,NaN,NaN,2016-07-05 13:16:38.200800,True
